In [1]:
import pandas as pd
import numpy as np
import requests
import os
import zipfile
import io
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')
import psycopg2
from sqlalchemy import create_engine

In [2]:
wb = pd.read_csv("ESGData.csv")
vdem = pd.read_csv('V-Dem-CY-Core-v13.csv')
country = pd.read_csv('ESGCountry.csv')

In [3]:
vdem = vdem[['country_text_id', 'country_name','year', 'v2x_polyarchy']]
vdem

,country_text_id,country_name,year,v2x_polyarchy
0,MEX,Mexico,1789,0.028
1,MEX,Mexico,1790,0.028
2,MEX,Mexico,1791,0.028
3,MEX,Mexico,1792,0.028
4,MEX,Mexico,1793,0.028
...,...,...,...,...
27550,SPD,Piedmont-Sardinia,1857,0.207
27551,SPD,Piedmont-Sardinia,1858,0.210
27552,SPD,Piedmont-Sardinia,1859,0.210
27553,SPD,Piedmont-Sardinia,1860,0.213


In [4]:
vdem = vdem.query('year >= 1960 and year <= 2021')

In [5]:
vdem

,country_text_id,country_name,year,v2x_polyarchy
171,MEX,Mexico,1960,0.232
172,MEX,Mexico,1961,0.234
173,MEX,Mexico,1962,0.233
174,MEX,Mexico,1963,0.233
175,MEX,Mexico,1964,0.231
...,...,...,...,...
26150,ZZB,Zanzibar,2017,0.267
26151,ZZB,Zanzibar,2018,0.268
26152,ZZB,Zanzibar,2019,0.266
26153,ZZB,Zanzibar,2020,0.258


In [6]:
vdem = vdem.rename(columns={"country_text_id": "country_code", "country_name": "country_name_vdem", "v2x_polyarchy": 'democracy'})

In [7]:
vdem = vdem.sort_values(by = ['year', 'country_code'], ascending = True)
vdem

,country_code,country_name_vdem,year,democracy
5433,AFG,Afghanistan,1960,0.080
15884,AGO,Angola,1960,0.009
1810,ALB,Albania,1960,0.176
5667,ARG,Argentina,1960,0.485
10084,AUS,Australia,1960,0.829
...,...,...,...,...
2272,YEM,Yemen,2021,0.114
1058,ZAF,South Africa,2021,0.702
9241,ZMB,Zambia,2021,0.411
9364,ZWE,Zimbabwe,2021,0.291


In [8]:
country = country[['Country Code', 'Table Name', 'Long Name', 'Currency Unit', 'Region','Income Group']]
country

,Country Code,Table Name,Long Name,Currency Unit,Region,Income Group
0,AFG,Afghanistan,Islamic State of Afghanistan,Afghan afghani,South Asia,Low income
1,AGO,Angola,People's Republic of Angola,Angolan kwanza,Sub-Saharan Africa,Lower middle income
2,ALB,Albania,Republic of Albania,Albanian lek,Europe & Central Asia,Upper middle income
3,AND,Andorra,Principality of Andorra,Euro,Europe & Central Asia,High income
4,ARB,Arab World,Arab World,NaN,NaN,NaN
...,...,...,...,...,...,...
234,WSM,Samoa,Samoa,Samoan tala,East Asia & Pacific,Lower middle income
235,YEM,"Yemen, Rep.",Republic of Yemen,Yemeni rial,Middle East & North Africa,Low income
236,ZAF,South Africa,Republic of South Africa,South African rand,Sub-Saharan Africa,Upper middle income
237,ZMB,Zambia,Republic of Zambia,New Zambian kwacha,Sub-Saharan Africa,Lower middle income


In [9]:
country = country.rename(columns={"Country Code": "country_code", "Table Name": "country_name_wb", "Long Name": 'country_longname', 'Currency Unit': 'currency_unit', 'Region': 'region', "Income Group": 'income_group'})
country

,country_code,country_name_wb,country_longname,currency_unit,region,income_group
0,AFG,Afghanistan,Islamic State of Afghanistan,Afghan afghani,South Asia,Low income
1,AGO,Angola,People's Republic of Angola,Angolan kwanza,Sub-Saharan Africa,Lower middle income
2,ALB,Albania,Republic of Albania,Albanian lek,Europe & Central Asia,Upper middle income
3,AND,Andorra,Principality of Andorra,Euro,Europe & Central Asia,High income
4,ARB,Arab World,Arab World,NaN,NaN,NaN
...,...,...,...,...,...,...
234,WSM,Samoa,Samoa,Samoan tala,East Asia & Pacific,Lower middle income
235,YEM,"Yemen, Rep.",Republic of Yemen,Yemeni rial,Middle East & North Africa,Low income
236,ZAF,South Africa,Republic of South Africa,South African rand,Sub-Saharan Africa,Upper middle income
237,ZMB,Zambia,Republic of Zambia,New Zambian kwacha,Sub-Saharan Africa,Lower middle income


In [10]:
noncountries = ["Arab World", "Central Europe and the Baltics",
"Caribbean small states",
"East Asia & Pacific (excluding high income)",
"Early-demographic dividend","East Asia & Pacific",
"Europe & Central Asia (excluding high income)",
"Europe & Central Asia", "Euro area",
"European Union","Fragile and conflict affected situations",
"High income",
"Heavily indebted poor countries (HIPC)","IBRD only",
"IDA & IBRD total",
"IDA total","IDA blend","IDA only",
"Latin America & Caribbean (excluding high income)",
"Latin America & Caribbean",
"Least developed countries: UN classification",
"Low income","Lower middle income","Low & middle income",
"Late-demographic dividend","Middle East & North Africa",
"Middle income",
"Middle East & North Africa (excluding high income)",
"North America","OECD members",
"Other small states","Pre-demographic dividend",
"Pacific island small states",
"Post-demographic dividend",
"Sub-Saharan Africa (excluding high income)",
"Sub-Saharan Africa",
"Small states","East Asia & Pacific (IDA & IBRD)",
"Europe & Central Asia (IDA & IBRD)",
"Latin America & Caribbean (IDA & IBRD)",
"Middle East & North Africa (IDA & IBRD)","South Asia",
"South Asia (IDA & IBRD)",
"Sub-Saharan Africa (IDA & IBRD)",
"Upper middle income", "World"]

In [11]:
noncountries

['Arab World',
 'Central Europe and the Baltics',
 'Caribbean small states',
 'East Asia & Pacific (excluding high income)',
 'Early-demographic dividend',
 'East Asia & Pacific',
 'Europe & Central Asia (excluding high income)',
 'Europe & Central Asia',
 'Euro area',
 'European Union',
 'Fragile and conflict affected situations',
 'High income',
 'Heavily indebted poor countries (HIPC)',
 'IBRD only',
 'IDA & IBRD total',
 'IDA total',
 'IDA blend',
 'IDA only',
 'Latin America & Caribbean (excluding high income)',
 'Latin America & Caribbean',
 'Least developed countries: UN classification',
 'Low income',
 'Lower middle income',
 'Low & middle income',
 'Late-demographic dividend',
 'Middle East & North Africa',
 'Middle income',
 'Middle East & North Africa (excluding high income)',
 'North America',
 'OECD members',
 'Other small states',
 'Pre-demographic dividend',
 'Pacific island small states',
 'Post-demographic dividend',
 'Sub-Saharan Africa (excluding high income)',
 'Sub

In [12]:
country = country.query('country_name_wb != @noncountries')

In [13]:
country

,country_code,country_name_wb,country_longname,currency_unit,region,income_group
0,AFG,Afghanistan,Islamic State of Afghanistan,Afghan afghani,South Asia,Low income
1,AGO,Angola,People's Republic of Angola,Angolan kwanza,Sub-Saharan Africa,Lower middle income
2,ALB,Albania,Republic of Albania,Albanian lek,Europe & Central Asia,Upper middle income
3,AND,Andorra,Principality of Andorra,Euro,Europe & Central Asia,High income
5,ARE,United Arab Emirates,United Arab Emirates,U.A.E. dirham,Middle East & North Africa,High income
...,...,...,...,...,...,...
234,WSM,Samoa,Samoa,Samoan tala,East Asia & Pacific,Lower middle income
235,YEM,"Yemen, Rep.",Republic of Yemen,Yemeni rial,Middle East & North Africa,Low income
236,ZAF,South Africa,Republic of South Africa,South African rand,Sub-Saharan Africa,Upper middle income
237,ZMB,Zambia,Republic of Zambia,New Zambian kwacha,Sub-Saharan Africa,Lower middle income


In [14]:
print(wb.columns)

Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
       '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022',
       'Unnamed: 67'],
      dtype='object')


In [15]:
wb

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 67
0,Arab World,ARB,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,86.705717,86.942778,87.228705,87.390856,87.617862,87.798740,87.948264,88.092536,NaN,NaN
1,Arab World,ARB,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,88.832276,89.053852,89.539016,90.662754,89.176939,90.352802,90.635050,90.845661,NaN,NaN
2,Arab World,ARB,Adjusted savings: natural resources depletion ...,NY.ADJ.DRES.GN.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,10.263684,6.217421,5.190735,6.458100,8.451699,7.351742,4.347213,NaN,NaN,NaN
3,Arab World,ARB,Adjusted savings: net forest depletion (% of GNI),NY.ADJ.DFOR.GN.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,0.084113,0.098458,0.093562,0.095137,0.050799,0.061584,0.077949,NaN,NaN,NaN
4,Arab World,ARB,Agricultural land (% of land area),AG.LND.AGRI.ZS,NaN,30.981414,30.982663,31.007054,31.018001,31.042466,...,39.834421,39.872575,39.937814,39.984452,39.969738,39.907031,39.973290,39.970742,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16964,Zimbabwe,ZWE,Terrestrial and marine protected areas (% of t...,ER.PTD.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,27.214542,27.214585,27.214585,27.214747,27.214747,27.214747,27.214747,NaN
16965,Zimbabwe,ZWE,Tree Cover Loss (hectares),AG.LND.FRLS.HA,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16966,Zimbabwe,ZWE,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,4.770000,5.282000,5.796000,6.279000,6.784000,7.370000,7.898000,8.067000,7.945000,NaN
16967,Zimbabwe,ZWE,Unmet need for contraception (% of married wom...,SP.UWT.TFRT,NaN,NaN,NaN,NaN,NaN,NaN,...,10.382129,10.400000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
wb = wb.drop(columns=['Indicator Name', 'Unnamed: 67'])

In [17]:
wb.columns

Index(['Country Name', 'Country Code', 'Indicator Code', '1960', '1961',
       '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970',
       '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979',
       '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988',
       '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997',
       '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006',
       '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015',
       '2016', '2017', '2018', '2019', '2020', '2021', '2022'],
      dtype='object')

In [18]:
wb = wb.rename(columns={"Country Code": "country_code", "Country Name": "country_name_wb", "Indicator Code": 'feature'})
wb

,country_name_wb,country_code,feature,1960,1961,1962,1963,1964,1965,1966,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Arab World,ARB,EG.CFT.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,86.406487,86.705717,86.942778,87.228705,87.390856,87.617862,87.798740,87.948264,88.092536,NaN
1,Arab World,ARB,EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,89.340705,88.832276,89.053852,89.539016,90.662754,89.176939,90.352802,90.635050,90.845661,NaN
2,Arab World,ARB,NY.ADJ.DRES.GN.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11.598544,10.263684,6.217421,5.190735,6.458100,8.451699,7.351742,4.347213,NaN,NaN
3,Arab World,ARB,NY.ADJ.DFOR.GN.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.062081,0.084113,0.098458,0.093562,0.095137,0.050799,0.061584,0.077949,NaN,NaN
4,Arab World,ARB,AG.LND.AGRI.ZS,NaN,30.981414,30.982663,31.007054,31.018001,31.042466,31.0504,...,39.838650,39.834421,39.872575,39.937814,39.984452,39.969738,39.907031,39.973290,39.970742,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16964,Zimbabwe,ZWE,ER.PTD.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27.214542,27.214585,27.214585,27.214747,27.214747,27.214747,27.214747
16965,Zimbabwe,ZWE,AG.LND.FRLS.HA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16966,Zimbabwe,ZWE,SL.UEM.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.994000,4.770000,5.282000,5.796000,6.279000,6.784000,7.370000,7.898000,8.067000,7.945000
16967,Zimbabwe,ZWE,SP.UWT.TFRT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,10.382129,10.400000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
noncountries.remove('World')
wb = wb.query('country_name_wb != @noncountries')

In [20]:
wb

,country_name_wb,country_code,feature,1960,1961,1962,1963,1964,1965,1966,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
3195,World,WLD,EG.CFT.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,61.095607,62.372600,63.662869,65.005668,66.321819,67.696517,68.921601,70.184805,71.331487,NaN
3196,World,WLD,EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,85.817366,86.306969,87.025241,88.192385,89.019548,89.893733,90.194716,90.483629,91.413941,NaN
3197,World,WLD,NY.ADJ.DRES.GN.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.778802,1.520162,0.820341,0.717655,0.919547,1.246028,1.087624,0.790953,NaN,NaN
3198,World,WLD,NY.ADJ.DFOR.GN.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.061422,0.069023,0.071087,0.073026,0.069027,0.048838,0.049834,0.058687,NaN,NaN
3199,World,WLD,AG.LND.AGRI.ZS,NaN,35.879317,35.95247,36.035383,36.117043,36.213941,36.294321,...,36.790232,36.789051,36.620496,36.587014,36.834001,36.738458,36.762648,36.730920,36.841665,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16964,Zimbabwe,ZWE,ER.PTD.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27.214542,27.214585,27.214585,27.214747,27.214747,27.214747,27.214747
16965,Zimbabwe,ZWE,AG.LND.FRLS.HA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16966,Zimbabwe,ZWE,SL.UEM.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.994000,4.770000,5.282000,5.796000,6.279000,6.784000,7.370000,7.898000,8.067000,7.945000
16967,Zimbabwe,ZWE,SP.UWT.TFRT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,10.382129,10.400000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
replace_map = {
"AG.LND.AGRI.ZS": "agricultural_land",
"AG.LND.FRST.ZS": "forest_area",
"AG.PRD.FOOD.XD": "food_production_index",
"CC.EST": "control_of_corruption",
"EG.CFT.ACCS.ZS": "access_to_clean_fuels_and_technologies_for_cooking",
"EG.EGY.PRIM.PP.KD": "energy_intensity_level_of_primary_energy",
"EG.ELC.ACCS.ZS": "access_to_electricity",
"EG.ELC.COAL.ZS": "electricity_production_from_coal_sources",
"EG.ELC.RNEW.ZS": "renewable_electricity_output",
"EG.FEC.RNEW.ZS": "renewable_energy_consumption",
"EG.IMP.CONS.ZS": "energy_imports",
"EG.USE.COMM.FO.ZS": "fossil_fuel_energy_consumption",
"EG.USE.PCAP.KG.OE": "energy_use",
"EN.ATM.CO2E.PC": "co2_emissions",
"EN.ATM.METH.PC": "methane_emissions",
"EN.ATM.NOXE.PC": "nitrous_oxide_emissions",
"EN.ATM.PM25.MC.M3": "pm2_5_air_pollution",
"EN.CLC.CDDY.XD": "cooling_degree_days",
"EN.CLC.GHGR.MT.CE": "ghg_net_emissions",
"EN.CLC.HEAT.XD": "heat_index_35",
"EN.CLC.MDAT.ZS": "droughts",
"EN.CLC.PRCP.XD": "maximum_5-day_rainfall",
"EN.CLC.SPEI.XD": "mean_drought_index",
"EN.MAM.THRD.NO":"mammal_species",
"EN.POP.DNST":"population_density",
"ER.H2O.FWTL.ZS":"annual_freshwater_withdrawals",
"ER.PTD.TOTL.ZS":"terrestrial_and_marine_protected_areas",
"GB.XPD.RSDV.GD.ZS":"research_and_development_expenditure",
"GE.EST":"government_effectiveness",
"IC.BUS.EASE.XQ":"ease_of_doing_business_rank",
"IC.LGL.CRED.XQ":"strength_of_legal_rights_index",
"IP.JRN.ARTC.SC":"scientific_and_technical_journal_articles",
"IP.PAT.RESD":"patent_applications",
"IT.NET.USER.ZS":"individuals_using_the_internet",
"NV.AGR.TOTL.ZS":"agriculture",
"NY.ADJ.DFOR.GN.ZS":"net_forest_depletion",
"NY.ADJ.DRES.GN.ZS":"natural_resources_depletion",
"NY.GDP.MKTP.KD.ZG":"gdp_growth",
"PV.EST":"political_stability_and_absence_of_violence",
"RL.EST":"rule_of_law",
"RQ.EST":"regulatory_quality",
"SE.ADT.LITR.ZS":"literacy_rate",
"SE.ENR.PRSC.FM.ZS":"gross_school_enrollment",
"SE.PRM.ENRR":"primary_school_enrollment",
"SE.XPD.TOTL.GB.ZS":"government_expenditure_on_education",
"SG.GEN.PARL.ZS":"proportion_of_seats_held_by_women_in_national_parliament",
"SH.DTH.COMM.ZS":"cause_of_death",
"SH.DYN.MORT":"mortality_rate",
"SH.H2O.SMDW.ZS":"people_using_safely_managed_drinking_water_services",
"SH.MED.BEDS.ZS":"hospital_beds",
"SH.STA.OWAD.ZS":"prevalence_of_overweight",
"SH.STA.SMSS.ZS":"people_using_safely_managed_sanitation_services",
"SI.DST.FRST.20":"income_share_held_by_lowest_20pct",
"SI.POV.GINI":"gini_index",
"SI.POV.NAHC":"poverty_headcount_ratio_at_national_poverty_lines",
"SI.SPR.PCAP.ZG":"annualized_average_growth_rate_in_per_capita_real_survey",
"SL.TLF.0714.ZS":"children_in_employment",
"SL.TLF.ACTI.ZS":"labor_force_participation_rate",
"SL.TLF.CACT.FM.ZS":"ratio_of_female_to_male_labor_force_participation_rate",
"SL.UEM.TOTL.ZS":"unemployment",
"SM.POP.NETM":"net_migration",
"SN.ITK.DEFC.ZS":"prevalence_of_undernourishment",
"SP.DYN.LE00.IN":"life_expectancy_at_birth",
"SP.DYN.TFRT.IN":"fertility_rate",
"SP.POP.65UP.TO.ZS":"population_ages_65_and_above",
"SP.UWT.TFRT":"unmet_need_for_contraception",
"VA.EST":"voice_and_accountability",
"EN.CLC.CSTP.ZS":"coastal_protection",
"SD.ESR.PERF.XQ":"economic_and_social_rights_performance_score",
"EN.CLC.HDDY.XD":"heating_degree_days",
"EN.LND.LTMP.DC":"land_surface_temperature",
"ER.H2O.FWST.ZS":"freshwater_withdrawal",
"EN.H2O.BDYS.ZS":"water_quality",
"AG.LND.FRLS.HA":"tree_cover_loss"
}


In [22]:
replace_map

{'AG.LND.AGRI.ZS': 'agricultural_land',
 'AG.LND.FRST.ZS': 'forest_area',
 'AG.PRD.FOOD.XD': 'food_production_index',
 'CC.EST': 'control_of_corruption',
 'EG.CFT.ACCS.ZS': 'access_to_clean_fuels_and_technologies_for_cooking',
 'EG.EGY.PRIM.PP.KD': 'energy_intensity_level_of_primary_energy',
 'EG.ELC.ACCS.ZS': 'access_to_electricity',
 'EG.ELC.COAL.ZS': 'electricity_production_from_coal_sources',
 'EG.ELC.RNEW.ZS': 'renewable_electricity_output',
 'EG.FEC.RNEW.ZS': 'renewable_energy_consumption',
 'EG.IMP.CONS.ZS': 'energy_imports',
 'EG.USE.COMM.FO.ZS': 'fossil_fuel_energy_consumption',
 'EG.USE.PCAP.KG.OE': 'energy_use',
 'EN.ATM.CO2E.PC': 'co2_emissions',
 'EN.ATM.METH.PC': 'methane_emissions',
 'EN.ATM.NOXE.PC': 'nitrous_oxide_emissions',
 'EN.ATM.PM25.MC.M3': 'pm2_5_air_pollution',
 'EN.CLC.CDDY.XD': 'cooling_degree_days',
 'EN.CLC.GHGR.MT.CE': 'ghg_net_emissions',
 'EN.CLC.HEAT.XD': 'heat_index_35',
 'EN.CLC.MDAT.ZS': 'droughts',
 'EN.CLC.PRCP.XD': 'maximum_5-day_rainfall',
 'EN.

In [23]:
wb = wb.replace(replace_map)
wb

,country_name_wb,country_code,feature,1960,1961,1962,1963,1964,1965,1966,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
3195,World,WLD,access_to_clean_fuels_and_technologies_for_coo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,61.095607,62.372600,63.662869,65.005668,66.321819,67.696517,68.921601,70.184805,71.331487,NaN
3196,World,WLD,access_to_electricity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,85.817366,86.306969,87.025241,88.192385,89.019548,89.893733,90.194716,90.483629,91.413941,NaN
3197,World,WLD,natural_resources_depletion,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.778802,1.520162,0.820341,0.717655,0.919547,1.246028,1.087624,0.790953,NaN,NaN
3198,World,WLD,net_forest_depletion,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.061422,0.069023,0.071087,0.073026,0.069027,0.048838,0.049834,0.058687,NaN,NaN
3199,World,WLD,agricultural_land,NaN,35.879317,35.95247,36.035383,36.117043,36.213941,36.294321,...,36.790232,36.789051,36.620496,36.587014,36.834001,36.738458,36.762648,36.730920,36.841665,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16964,Zimbabwe,ZWE,terrestrial_and_marine_protected_areas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27.214542,27.214585,27.214585,27.214747,27.214747,27.214747,27.214747
16965,Zimbabwe,ZWE,tree_cover_loss,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16966,Zimbabwe,ZWE,unemployment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.994000,4.770000,5.282000,5.796000,6.279000,6.784000,7.370000,7.898000,8.067000,7.945000
16967,Zimbabwe,ZWE,unmet_need_for_contraception,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,10.382129,10.400000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
wb.columns

Index(['country_name_wb', 'country_code', 'feature', '1960', '1961', '1962',
       '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1971',
       '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980',
       '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989',
       '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998',
       '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007',
       '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016',
       '2017', '2018', '2019', '2020', '2021', '2022'],
      dtype='object')

In [25]:
wb = pd.melt(wb, id_vars=['country_name_wb', 'country_code'
                         ,'feature'], var_name='year', value_name='value')

In [26]:
wb

,country_name_wb,country_code,feature,year,value
0,World,WLD,access_to_clean_fuels_and_technologies_for_coo...,1960,NaN
1,World,WLD,access_to_electricity,1960,NaN
2,World,WLD,natural_resources_depletion,1960,NaN
3,World,WLD,net_forest_depletion,1960,NaN
4,World,WLD,agricultural_land,1960,NaN
...,...,...,...,...,...
867757,Zimbabwe,ZWE,terrestrial_and_marine_protected_areas,2022,27.214747
867758,Zimbabwe,ZWE,tree_cover_loss,2022,NaN
867759,Zimbabwe,ZWE,unemployment,2022,7.945000
867760,Zimbabwe,ZWE,unmet_need_for_contraception,2022,NaN


In [27]:
wb = wb.pivot_table(index=['country_name_wb','country_code', 'year'], columns='feature', values='value').reset_index()
 
 
wb.columns.name = None
 
 
wb = wb.reset_index(drop=True)
 
wb.head()

,country_name_wb,country_code,year,access_to_clean_fuels_and_technologies_for_cooking,access_to_electricity,agricultural_land,agriculture,annual_freshwater_withdrawals,annualized_average_growth_rate_in_per_capita_real_survey,cause_of_death,...,research_and_development_expenditure,rule_of_law,scientific_and_technical_journal_articles,strength_of_legal_rights_index,terrestrial_and_marine_protected_areas,tree_cover_loss,unemployment,unmet_need_for_contraception,voice_and_accountability,water_quality
0,Afghanistan,AFG,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,1961,NaN,NaN,57.878356,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,AFG,1962,NaN,NaN,57.955016,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,AFG,1963,NaN,NaN,58.031676,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,AFG,1964,NaN,NaN,58.116002,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
wb['year'] = pd.to_numeric(wb['year'])
wb.dtypes

country_name_wb                                        object
country_code                                           object
year                                                    int64
access_to_clean_fuels_and_technologies_for_cooking    float64
access_to_electricity                                 float64
                                                       ...   
tree_cover_loss                                       float64
unemployment                                          float64
unmet_need_for_contraception                          float64
voice_and_accountability                              float64
water_quality                                         float64
Length: 74, dtype: object

In [29]:
newdf = wb[wb['country_name_wb'] == 'World']

In [30]:
newdf

,country_name_wb,country_code,year,access_to_clean_fuels_and_technologies_for_cooking,access_to_electricity,agricultural_land,agriculture,annual_freshwater_withdrawals,annualized_average_growth_rate_in_per_capita_real_survey,cause_of_death,...,research_and_development_expenditure,rule_of_law,scientific_and_technical_journal_articles,strength_of_legal_rights_index,terrestrial_and_marine_protected_areas,tree_cover_loss,unemployment,unmet_need_for_contraception,voice_and_accountability,water_quality
11970,World,WLD,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11971,World,WLD,1961,NaN,NaN,35.879317,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11972,World,WLD,1962,NaN,NaN,35.952470,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11973,World,WLD,1963,NaN,NaN,36.035383,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11974,World,WLD,1964,NaN,NaN,36.117043,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12028,World,WLD,2018,67.696517,89.893733,36.738458,3.951747,9.137864,NaN,NaN,...,2.196050,NaN,2.565871e+06,5.507853,14.337515,NaN,5.699076,NaN,NaN,NaN
12029,World,WLD,2019,68.921601,90.194716,36.762648,4.013242,9.167900,NaN,18.40885,...,2.331934,NaN,2.734962e+06,5.685864,14.599801,NaN,5.543161,NaN,NaN,NaN
12030,World,WLD,2020,70.184805,90.483629,36.730920,4.352489,9.109511,NaN,NaN,...,2.627081,NaN,2.933011e+06,NaN,14.787111,NaN,6.898273,NaN,NaN,NaN
12031,World,WLD,2021,71.331487,91.413941,36.841665,4.304093,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,14.552260,NaN,6.201871,NaN,NaN,NaN


In [31]:
test = wb[wb['country_name_wb'] != 'World']

In [32]:
test

,country_name_wb,country_code,year,access_to_clean_fuels_and_technologies_for_cooking,access_to_electricity,agricultural_land,agriculture,annual_freshwater_withdrawals,annualized_average_growth_rate_in_per_capita_real_survey,cause_of_death,...,research_and_development_expenditure,rule_of_law,scientific_and_technical_journal_articles,strength_of_legal_rights_index,terrestrial_and_marine_protected_areas,tree_cover_loss,unemployment,unmet_need_for_contraception,voice_and_accountability,water_quality
0,Afghanistan,AFG,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,1961,NaN,NaN,57.878356,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,AFG,1962,NaN,NaN,57.955016,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,AFG,1963,NaN,NaN,58.031676,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,AFG,1964,NaN,NaN,58.116002,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12217,Zimbabwe,ZWE,2018,30.0,45.400288,41.876696,7.319375,30.761677,NaN,NaN,...,NaN,-1.292463,406.23,5.0,27.214585,NaN,6.784,NaN,-1.136798,NaN
12218,Zimbabwe,ZWE,2019,30.2,46.682095,41.876696,9.819262,30.761677,NaN,47.647301,...,NaN,-1.303515,431.62,6.0,27.214747,NaN,7.370,NaN,-1.163669,NaN
12219,Zimbabwe,ZWE,2020,30.3,52.747667,41.876696,8.772859,30.761677,NaN,NaN,...,NaN,-1.329611,480.16,NaN,27.214747,NaN,7.898,NaN,-1.113408,83.3
12220,Zimbabwe,ZWE,2021,30.3,48.979927,41.876696,8.849899,NaN,NaN,NaN,...,NaN,-1.277202,NaN,NaN,27.214747,NaN,8.067,NaN,-1.135830,NaN


In [33]:
worlddf = newdf.drop(columns=['country_code', 'country_name_wb'])
worlddf

,year,access_to_clean_fuels_and_technologies_for_cooking,access_to_electricity,agricultural_land,agriculture,annual_freshwater_withdrawals,annualized_average_growth_rate_in_per_capita_real_survey,cause_of_death,children_in_employment,co2_emissions,...,research_and_development_expenditure,rule_of_law,scientific_and_technical_journal_articles,strength_of_legal_rights_index,terrestrial_and_marine_protected_areas,tree_cover_loss,unemployment,unmet_need_for_contraception,voice_and_accountability,water_quality
11970,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11971,1961,NaN,NaN,35.879317,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11972,1962,NaN,NaN,35.952470,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11973,1963,NaN,NaN,36.035383,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11974,1964,NaN,NaN,36.117043,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12028,2018,67.696517,89.893733,36.738458,3.951747,9.137864,NaN,NaN,NaN,4.641294,...,2.196050,NaN,2.565871e+06,5.507853,14.337515,NaN,5.699076,NaN,NaN,NaN
12029,2019,68.921601,90.194716,36.762648,4.013242,9.167900,NaN,18.40885,NaN,4.582036,...,2.331934,NaN,2.734962e+06,5.685864,14.599801,NaN,5.543161,NaN,NaN,NaN
12030,2020,70.184805,90.483629,36.730920,4.352489,9.109511,NaN,NaN,NaN,4.291853,...,2.627081,NaN,2.933011e+06,NaN,14.787111,NaN,6.898273,NaN,NaN,NaN
12031,2021,71.331487,91.413941,36.841665,4.304093,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,14.552260,NaN,6.201871,NaN,NaN,NaN


In [34]:
worlddf.columns = ['world_' + column if column != 'year' else column for column in worlddf.columns]
worlddf

,year,world_access_to_clean_fuels_and_technologies_for_cooking,world_access_to_electricity,world_agricultural_land,world_agriculture,world_annual_freshwater_withdrawals,world_annualized_average_growth_rate_in_per_capita_real_survey,world_cause_of_death,world_children_in_employment,world_co2_emissions,...,world_research_and_development_expenditure,world_rule_of_law,world_scientific_and_technical_journal_articles,world_strength_of_legal_rights_index,world_terrestrial_and_marine_protected_areas,world_tree_cover_loss,world_unemployment,world_unmet_need_for_contraception,world_voice_and_accountability,world_water_quality
11970,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11971,1961,NaN,NaN,35.879317,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11972,1962,NaN,NaN,35.952470,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11973,1963,NaN,NaN,36.035383,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11974,1964,NaN,NaN,36.117043,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12028,2018,67.696517,89.893733,36.738458,3.951747,9.137864,NaN,NaN,NaN,4.641294,...,2.196050,NaN,2.565871e+06,5.507853,14.337515,NaN,5.699076,NaN,NaN,NaN
12029,2019,68.921601,90.194716,36.762648,4.013242,9.167900,NaN,18.40885,NaN,4.582036,...,2.331934,NaN,2.734962e+06,5.685864,14.599801,NaN,5.543161,NaN,NaN,NaN
12030,2020,70.184805,90.483629,36.730920,4.352489,9.109511,NaN,NaN,NaN,4.291853,...,2.627081,NaN,2.933011e+06,NaN,14.787111,NaN,6.898273,NaN,NaN,NaN
12031,2021,71.331487,91.413941,36.841665,4.304093,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,14.552260,NaN,6.201871,NaN,NaN,NaN


In [35]:
# I believe that the merge is going to many to many

merge = pd.merge(test, vdem, on = ['country_code', 'year'],
                  how = 'outer', validate = 'many_to_many', indicator= 'matched')

In [36]:
merge.value_counts('matched')

matched
both          9976
left_only     2183
right_only     395
Name: count, dtype: int64

In [37]:
wb_only = merge.query('matched == "left_only"')

In [38]:
wb_only

,country_name_wb,country_code,year,access_to_clean_fuels_and_technologies_for_cooking,access_to_electricity,agricultural_land,agriculture,annual_freshwater_withdrawals,annualized_average_growth_rate_in_per_capita_real_survey,cause_of_death,...,strength_of_legal_rights_index,terrestrial_and_marine_protected_areas,tree_cover_loss,unemployment,unmet_need_for_contraception,voice_and_accountability,water_quality,country_name_vdem,democracy,matched
62,Afghanistan,AFG,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3.637257,NaN,NaN,NaN,-1.751587,NaN,NaN,NaN,left_only
125,Albania,ALB,2022,NaN,NaN,NaN,18.628032,NaN,NaN,NaN,...,NaN,14.175197,NaN,11.808,NaN,0.139466,NaN,NaN,NaN,left_only
188,Algeria,DZA,2022,NaN,NaN,NaN,11.384084,NaN,NaN,NaN,...,NaN,4.400036,NaN,11.550,NaN,-1.003874,NaN,NaN,NaN,left_only
189,Andorra,AND,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
190,Andorra,AND,1961,NaN,NaN,55.319149,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11906,"Venezuela, RB",VEN,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,39.000859,NaN,5.328,NaN,-1.533926,NaN,NaN,NaN,left_only
11969,Vietnam,VNM,2022,NaN,NaN,NaN,11.877108,NaN,NaN,NaN,...,NaN,2.929507,NaN,1.923,NaN,-1.288703,NaN,NaN,NaN,left_only
12032,"Yemen, Rep.",YEM,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.605801,NaN,13.590,NaN,-1.618616,NaN,NaN,NaN,left_only
12095,Zambia,ZMB,2022,NaN,NaN,NaN,3.388658,NaN,NaN,NaN,...,NaN,41.259399,NaN,6.131,NaN,-0.089141,NaN,NaN,NaN,left_only


In [39]:
x = wb_only.groupby(['country_code', 'country_name_wb'])['year'].agg(['min', 'max'])

In [40]:
x.head(200)

,,min,max
country_code,country_name_wb,,
AFG,Afghanistan,2022,2022
AGO,Angola,2022,2022
ALB,Albania,2022,2022
AND,Andorra,1960,2022
ARE,United Arab Emirates,1960,2022
...,...,...,...
WSM,Samoa,1960,2022
YEM,"Yemen, Rep.",2022,2022
ZAF,South Africa,2022,2022


In [41]:
with pd.option_context('display.max_rows', None,):
    print(x)

                                              min   max
country_code country_name_wb                           
AFG          Afghanistan                     2022  2022
AGO          Angola                          2022  2022
ALB          Albania                         2022  2022
AND          Andorra                         1960  2022
ARE          United Arab Emirates            1960  2022
ARG          Argentina                       2022  2022
ARM          Armenia                         1960  2022
ATG          Antigua and Barbuda             1960  2022
AUS          Australia                       2022  2022
AUT          Austria                         2022  2022
AZE          Azerbaijan                      1960  2022
BDI          Burundi                         2022  2022
BEL          Belgium                         2022  2022
BEN          Benin                           2022  2022
BFA          Burkina Faso                    2022  2022
BGD          Bangladesh                      196

In [42]:
vdem_only = merge.query('matched == "right_only"')
y = vdem_only.groupby(['country_code', 'country_name_vdem'])['year'].agg(['min', 'max'])

In [43]:
y

,,min,max
country_code,country_name_vdem,,
DDR,German Democratic Republic,1960,1990
HKG,Hong Kong,1960,2021
PSE,Palestine/West Bank,1967,2021
PSG,Palestine/Gaza,1960,2021
SML,Somaliland,1991,2021
TWN,Taiwan,1960,2021
VDR,Republic of Vietnam,1960,1975
XKX,Kosovo,1999,2021
YMD,South Yemen,1960,1990


In [44]:
new_merge = pd.merge(test, vdem, on = ['country_code', 'year'],
                  how = 'inner')

In [45]:
new_merge

,country_name_wb,country_code,year,access_to_clean_fuels_and_technologies_for_cooking,access_to_electricity,agricultural_land,agriculture,annual_freshwater_withdrawals,annualized_average_growth_rate_in_per_capita_real_survey,cause_of_death,...,scientific_and_technical_journal_articles,strength_of_legal_rights_index,terrestrial_and_marine_protected_areas,tree_cover_loss,unemployment,unmet_need_for_contraception,voice_and_accountability,water_quality,country_name_vdem,democracy
0,Afghanistan,AFG,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Afghanistan,0.080
1,Afghanistan,AFG,1961,NaN,NaN,57.878356,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Afghanistan,0.083
2,Afghanistan,AFG,1962,NaN,NaN,57.955016,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Afghanistan,0.082
3,Afghanistan,AFG,1963,NaN,NaN,58.031676,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Afghanistan,0.085
4,Afghanistan,AFG,1964,NaN,NaN,58.116002,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Afghanistan,0.137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9971,Zimbabwe,ZWE,2017,29.8,43.979065,41.876696,8.340969,27.234910,-3.5,NaN,...,334.71,5.0,27.214585,NaN,6.279,NaN,-1.195905,76.5,Zimbabwe,0.295
9972,Zimbabwe,ZWE,2018,30.0,45.400288,41.876696,7.319375,30.761677,NaN,NaN,...,406.23,5.0,27.214585,NaN,6.784,NaN,-1.136798,NaN,Zimbabwe,0.305
9973,Zimbabwe,ZWE,2019,30.2,46.682095,41.876696,9.819262,30.761677,NaN,47.647301,...,431.62,6.0,27.214747,NaN,7.370,NaN,-1.163669,NaN,Zimbabwe,0.293
9974,Zimbabwe,ZWE,2020,30.3,52.747667,41.876696,8.772859,30.761677,NaN,NaN,...,480.16,NaN,27.214747,NaN,7.898,NaN,-1.113408,83.3,Zimbabwe,0.293


In [46]:
# newmerge, worlddf, country
new_merge.to_csv("timeseries.csv")
worlddf.to_csv("world.csv")
country.to_csv("country.csv")
# timeseries has country name twice, fails
# timeseries is all atomic, PK will be country name and year.. country code just depends on countryname
# world is all atomic, PK is just the year
# country is all atomic, PK is country
# I believe that these three tables
# 1st one failure
# 2nd pass
# 3rd pass

In [47]:
timeseries = new_merge.drop(columns = ['country_name_vdem'])

In [48]:
dbserver = psycopg2.connect(
    host = 'postgres',
    user = 'postgres',
    password = POSTGRES_PASSWORD,
    port = 5432
)
dbserver.autocommit = True

In [49]:
cursor = dbserver.cursor()

In [50]:
try:
    cursor.execute('CREATE DATABASE cardib')
except:
    cursor.execute('DROP DATABASE cardib')
    cursor.execute('CREATE DATABASE cardib')

In [51]:
engine = create_engine('postgresql+psycopg2://{user}:{password}@{host}:{port}/{db}'.format(
    user = 'postgres',
    password = POSTGRES_PASSWORD,
    host = 'postgres',
    port = 5432,
    db = 'cardib'
))

In [52]:
timeseries.to_sql('timeseries', con=engine, index=False, chunksize=500, if_exists = 'replace')

1256

In [53]:
worlddf.to_sql('world', con=engine, index=False, chunksize=1000, if_exists = 'replace')

63

In [54]:
country.to_sql('country', con=engine, index=False, chunksize=1000, if_exists = 'replace')

193

In [81]:
for col in timeseries.columns:
    print(col, timeseries[col].dtype)

country_name_wb object
country_code object
year int64
access_to_clean_fuels_and_technologies_for_cooking float64
access_to_electricity float64
agricultural_land float64
agriculture float64
annual_freshwater_withdrawals float64
annualized_average_growth_rate_in_per_capita_real_survey float64
cause_of_death float64
children_in_employment float64
co2_emissions float64
coastal_protection float64
control_of_corruption float64
cooling_degree_days float64
economic_and_social_rights_performance_score float64
electricity_production_from_coal_sources float64
energy_imports float64
energy_intensity_level_of_primary_energy float64
energy_use float64
fertility_rate float64
food_production_index float64
forest_area float64
fossil_fuel_energy_consumption float64
freshwater_withdrawal float64
gdp_growth float64
ghg_net_emissions float64
gini_index float64
government_effectiveness float64
government_expenditure_on_education float64
gross_school_enrollment float64
heat_index_35 float64
heating_degree_da

In [80]:
for col in worlddf.columns:
    print(col, worlddf[col].dtype)

year int64
world_access_to_clean_fuels_and_technologies_for_cooking float64
world_access_to_electricity float64
world_agricultural_land float64
world_agriculture float64
world_annual_freshwater_withdrawals float64
world_annualized_average_growth_rate_in_per_capita_real_survey float64
world_cause_of_death float64
world_children_in_employment float64
world_co2_emissions float64
world_coastal_protection float64
world_control_of_corruption float64
world_cooling_degree_days float64
world_economic_and_social_rights_performance_score float64
world_electricity_production_from_coal_sources float64
world_energy_imports float64
world_energy_intensity_level_of_primary_energy float64
world_energy_use float64
world_fertility_rate float64
world_food_production_index float64
world_forest_area float64
world_fossil_fuel_energy_consumption float64
world_freshwater_withdrawal float64
world_gdp_growth float64
world_ghg_net_emissions float64
world_gini_index float64
world_government_effectiveness float64
wo

In [57]:
country.dtypes

country_code        object
country_name_wb     object
country_longname    object
currency_unit       object
region              object
income_group        object
dtype: object

In [59]:
myquery = '''
SELECT t.democracy, t.country_name_wb AS country
FROM timeseries t
INNER JOIN country c
    ON t.country_name_wb = c.country_name_wb
WHERE year = 2021
ORDER BY t.democracy DESC
LIMIT 15
'''

pd.read_sql_query(myquery, con=engine)

,democracy,country
0,0.915,Denmark
1,0.903,Sweden
2,0.901,Norway
3,0.898,Switzerland
4,0.898,Costa Rica
5,0.896,Estonia
6,0.895,Ireland
7,0.893,Belgium
8,0.888,Portugal
9,0.883,New Zealand


In [60]:
myquery = '''
SELECT t.life_expectancy_at_birth, t.country_name_wb, w.world_life_expectancy_at_birth, w.year
FROM timeseries t
FULL JOIN world w
    ON t.year = w.year
WHERE t.country_code = 'CHL'
ORDER BY t.year
'''

pd.read_sql_query(myquery, con=engine)

,life_expectancy_at_birth,country_name_wb,world_life_expectancy_at_birth,year
0,57.015,Chile,50.894180,1960
1,57.537,Chile,52.846336,1961
2,57.771,Chile,55.208684,1962
3,57.150,Chile,55.542341,1963
4,58.738,Chile,56.034875,1964
...,...,...,...,...
57,80.350,Chile,72.542776,2017
58,80.133,Chile,72.784090,2018
59,80.326,Chile,72.979716,2019
60,79.377,Chile,72.243822,2020


In [61]:
#### not done

myquery = '''
SELECT c.region, t.co2_emissions
FROM timeseries t
FULL JOIN country c
    ON t.country_name_wb = c.country_name_wb
WHERE year = 2019
ORDER BY t.year
'''

pd.read_sql_query(myquery, con=engine)

,region,co2_emissions
0,South Asia,0.297564
1,Europe & Central Asia,1.749462
2,Middle East & North Africa,3.994402
3,Sub-Saharan Africa,0.753638
4,Latin America & Caribbean,3.742030
...,...,...
167,Latin America & Caribbean,3.710599
168,East Asia & Pacific,3.567848
169,Middle East & North Africa,0.354864
170,Sub-Saharan Africa,0.414336


In [62]:
myquery = '''
WITH old_democracy AS (
  SELECT country_code, democracy
  FROM timeseries
  WHERE year = 1960
),
new_democracy AS (
  SELECT country_code, democracy
  FROM timeseries
  WHERE year = 2021
)
SELECT c.country_name_wb,
       old_democracy.democracy AS old_demo,
       new_democracy.democracy AS new_demo,
       (new_democracy.democracy - old_democracy.democracy) AS difference
FROM country c
JOIN old_democracy ON c.country_code = old_democracy.country_code
JOIN new_democracy ON c.country_code = new_democracy.country_code
ORDER BY difference DESC
LIMIT 10;
'''

pd.read_sql_query(myquery, con=engine)


,country_name_wb,old_demo,new_demo,difference
0,Spain,0.070,0.854,0.784
1,Portugal,0.128,0.888,0.760
2,Cabo Verde,0.023,0.773,0.750
3,Vanuatu,0.080,0.771,0.691
4,Timor-Leste,0.018,0.680,0.662
5,São Tomé and Principe,0.030,0.690,0.660
6,Romania,0.158,0.768,0.610
7,Czechia,0.196,0.805,0.609
8,Seychelles,0.130,0.714,0.584
9,Malawi,0.040,0.619,0.579


In [76]:
##### find the most used currency by counting how many countries use it 

myquery = '''
SELECT c.currency_unit, COUNT(c.country_name_wb) AS count
FROM country c
GROUP BY c.currency_unit
ORDER BY count DESC
'''

pd.read_sql_query(myquery, con=engine)

,currency_unit,count
0,Euro,24
1,West African CFA franc,8
2,U.S. dollar,7
3,East Caribbean dollar,6
4,Central African CFA franc,6
...,...,...
138,Lao kip,1
139,Czech koruna,1
140,Iraqi dinar,1
141,New Zambian kwacha,1


In [66]:
### income group - country
### one country - country
### GINI index timeseries
## same country multiple times

myquery = '''
SELECT AVG(t.gini_index), c.income_group

FROM country c
INNER JOIN timeseries t
    ON c.country_name_wb = t.country_name_wb
GROUP BY c.income_group
'''

pd.read_sql_query(myquery, con=engine)

,avg,income_group
0,33.145046,High income
1,39.883333,Lower middle income
2,41.648421,Low income
3,48.007692,None
4,42.653654,Upper middle income
